# Intro to Polars with NYC taxi data

In [3]:
import polars as pl

## Parquet vs Arrow

In [4]:
df = pl.read_parquet("data/fhvhv_data.parquet")

In [6]:
df.write_ipc("data/fhvhv_data.arrow")

In [13]:
df = pl.read_ipc("data/fhvhv_data.arrow")
df2 = df.select('hvfhs_license_num', 'dispatching_base_num', 'tolls', 'sales_tax','tips', 'trip_miles', 'trip_time')
df2

hvfhs_license_num,dispatching_base_num,tolls,sales_tax,tips,trip_miles,trip_time
str,str,f64,f64,f64,f64,i64
"""HV0003""","""B03404""",0.0,2.21,0.0,1.18,664
"""HV0003""","""B03404""",0.0,1.06,0.0,0.82,460
"""HV0003""","""B03404""",0.0,2.65,0.0,1.18,595
"""HV0003""","""B03404""",0.0,0.7,0.0,1.65,303
"""HV0003""","""B03404""",0.0,0.84,0.0,1.65,461
"""HV0003""","""B03404""",0.0,1.57,0.0,4.51,762
"""HV0003""","""B03404""",0.0,1.48,0.0,3.68,931
"""HV0003""","""B03404""",0.0,1.28,4.0,2.77,843
"""HV0003""","""B03404""",0.0,0.94,0.0,2.04,710


In [14]:
df2.filter(pl.col('tips') > 5)

hvfhs_license_num,dispatching_base_num,tolls,sales_tax,tips,trip_miles,trip_time
str,str,f64,f64,f64,f64,i64
"""HV0003""","""B03404""",0.0,3.45,10.0,3.93,1389
"""HV0003""","""B03404""",0.0,2.03,5.11,10.05,1362
"""HV0003""","""B03404""",0.0,3.14,7.92,7.92,1506
"""HV0003""","""B03404""",0.0,1.78,5.03,1.86,515
"""HV0003""","""B03404""",0.0,2.73,5.1,5.07,2088
"""HV0003""","""B03404""",0.0,0.76,10.0,0.96,407
"""HV0003""","""B03404""",0.0,2.43,5.01,3.03,797
"""HV0003""","""B03404""",6.55,2.83,9.62,8.29,1137
"""HV0005""","""B03406""",4.44,3.26,8.77,13.445,2139


In [15]:
df2.with_columns(pl.col("trip_miles").round(1).alias("rounded_miles"))

hvfhs_license_num,dispatching_base_num,tolls,sales_tax,tips,trip_miles,trip_time,rounded_miles
str,str,f64,f64,f64,f64,i64,f64
"""HV0003""","""B03404""",0.0,2.21,0.0,1.18,664,1.2
"""HV0003""","""B03404""",0.0,1.06,0.0,0.82,460,0.8
"""HV0003""","""B03404""",0.0,2.65,0.0,1.18,595,1.2
"""HV0003""","""B03404""",0.0,0.7,0.0,1.65,303,1.7
"""HV0003""","""B03404""",0.0,0.84,0.0,1.65,461,1.7
"""HV0003""","""B03404""",0.0,1.57,0.0,4.51,762,4.5
"""HV0003""","""B03404""",0.0,1.48,0.0,3.68,931,3.7
"""HV0003""","""B03404""",0.0,1.28,4.0,2.77,843,2.8
"""HV0003""","""B03404""",0.0,0.94,0.0,2.04,710,2.0


In [21]:
df.select('hvfhs_license_num', 'dispatching_base_num', 'tolls', 'sales_tax','tips', 'trip_miles', 'trip_time').with_columns(
            [
                pl.col("trip_miles")
                .round(0)
                .alias("rounded_miles"),
                pl.col("trip_time")
                .apply(lambda x: round(x / 100, 0) * 100)
                .alias("rounded_time"),
            ]
        ).groupby(["rounded_miles", "rounded_time"]).count()

rounded_miles,rounded_time,count
f64,f64,u32
95.0,8300.0,3
96.0,12300.0,1
114.0,9300.0,1
122.0,10400.0,1
91.0,13200.0,1
46.0,10900.0,2
51.0,8400.0,1
8.0,22200.0,1
56.0,8500.0,2


In [22]:
ldf = pl.scan_ipc("data/fhvhv_data.arrow")
ldf.select('hvfhs_license_num', 'dispatching_base_num', 'tolls', 'sales_tax','tips', 'trip_miles', 'trip_time').with_columns(
            [
                pl.col("trip_miles")
                .round(0)
                .alias("rounded_miles"),
                pl.col("trip_time")
                .apply(lambda x: round(x / 100, 0) * 100)
                .alias("rounded_time"),
            ]
        ).groupby(["rounded_miles", "rounded_time"]).count().collect()

rounded_miles,rounded_time,count
f64,f64,u32
95.0,8300.0,3
96.0,12300.0,1
114.0,9300.0,1
122.0,10400.0,1
91.0,13200.0,1
46.0,10900.0,2
51.0,8400.0,1
8.0,22200.0,1
56.0,8500.0,2


In [5]:
ldf.select([pl.col("tips"), pl.col("driver_pay")]).fetch()

tips,driver_pay
f64,f64
0.0,23.03
0.0,12.32
0.0,23.3
0.0,6.3
0.0,7.44
0.0,12.25
0.0,12.75
4.0,11.47
0.0,9.55
